# "Document Retrieval": Heraussuchen passender Dokumente aus einer großen Datenbank

Wie sie im letzten Notebook gesehen haben, sind auf Question-Answering-Datensätzen ([SQuAD](https://rajpurkar.github.io/SQuAD-explorer/), [MLQA](https://github.com/facebookresearch/MLQA), [XQuAD](https://github.com/deepmind/xquad)) [nachtrainierte](https://huggingface.co/Sahajtomar/GBERTQnA) [BERT](https://ai.googleblog.com/2018/11/open-sourcing-bert-state-of-art-pre.html)-Modelle, schon recht gut darin, aus kurzen Textpassagen relevante Information zu extrahieren. Allerdings ist die Größe der Textblöcke, die auf einmal verarbeitet werden können, noch durch die verfügbare Zeit, Rechenkapazität und den Arbeitsspeicher limitiert. Meist können nur bis zu 512 "Tokens" (d.h. Wörter, Wortteile oder Buchstaben) auf einmal verarbeitet werden. Zudem können einzelne Aufrufe eines Transformermodells einige Sekunden dauern, so dass es bis jetzt noch nicht möglich ist, eine komplette Wissensdatenbank (z.B. die deutsche Wikipedia mit ihren mehr als 2,5 Millionen Artikeln) allein mit solchen Modellen zu durchsuchen. 

Deshalb ist es oft unumgänglich, **vor** den Aufruf des trainierten Transformermodells noch eine konventionelle Suchmaschinen-Logik zu stellen, die anhand der gestellten Frage zunächst aus einem großen Dokumentenkorpus (z.B. den Artikeln der deutschen Wikipedia) die passenden Dokumente (z.B. einzelne Wikipediaartikel) heraussucht. D.h. die Suchlogik muss in der Lage sein, passende Dokumente bereitzustellen, die kurz genug sind, dass sie ein Transformer auf einmal (oder wie wir unten sehen werden, in wenige "mundgerechte" Stücke zerlegt) verarbeiten kann. Da auf (Extractive!)-Question-Answering trainierte Transformer-Netzwerke jedoch nur Antworten finden können, die in dem mitgelieferten Artikel bzw. Textabschnitt enthalten sind, bedeutet das gleichzeitig, dass das Heraussuchen der passenden Dokumente bereits ein Faktor ist, der die Qualität der möglichen Antworten stark einschränken kann. Wenn ich dem Transformer nämlich nur unpassende Artikel anbiete, die gar nichts mit der Frage zu tun haben, wird das Netzwerk darin natürlich auch keine Antwort finden können.

Damit sie selbst erfahren können, wie solch ein System funktionieren (bzw. auch nicht funktionieren/scheitern) kann, möchten wir ihnen ein mögliches Vorgehen in den folgenden Zellen anhand eines Beispiels demonstrieren. Dabei werden wir gemeinsam ein System entwickeln, das versucht deutsche Fragen in natürlicher Sprache anhand der deutschen Wikipedia zu beantworten.

## Aufgabe

- __Lesen__ sie die entsprechenden Texte, __führen__ sie die zugehörigen __Codezellen aus__ und bearbeiten sie die __eingestreuten Aufgaben__.
- __Dokumentieren und kommentieren__ sie wieder interessante, bemerkenswerte oder überraschende Ergebnisse (z.B. sehr gut passende oder unpassende Artikelvorschläge) zusammen mit den entsprechenden Fragetexten in dem __zugehörigen Miro-Board__, welches in den geteilten Notizen verlinkt ist.

## Fragen beantworten auf Grundlage der deutschen Wikipedia

Glücklicher Weise steht für den Zugriff auf die Wikipedia tatsächlich ein sehr gutes Python-Paket zur Verfügung, welches wir in der nächsten Zelle importieren.

In [ ]:
# Importiere das wikipedia-Paket  als "wiki"
import wikipedia as wiki

# Setze die Sprache der Wikipedia auf "de"utsch
wiki.set_lang('de')

## Auf Grundlage der Frage einen passenden Wikipedia-Artikel heraussuchen

Zunächst definieren wir die Frage, die beantwortet werden soll.

In [ ]:
frage = "Wie viele Bundesländer hat die Bundesrepublik Deutschland?"

Zum Glück müssen wir uns nicht darum kümmern, wie wir Wikipedia-Artikel finden, die zu dieser Frage passen, da die Wikipedia selbst eine sehr gute Suchfunktion hat, auf die wir mit dem "wiki"-Paket zugreifen können.

In der folgenden Zelle rufen wir die Wikipediasuche auf und lassen uns eine Liste mit passenden Artikeln zurückgeben.

In [ ]:
# Mit wiki.search können wir die Wikipedia-Suchfunktion aufrufen.
# Der Parameter "results" gibt dabei an, wie viele Artikel-
# vorschläge gesucht werden sollen.
# Diese werden von der Funktion als Liste
# zurückgegeben.
suchergebnisse = wiki.search(frage, results = 3)

print('Liste der vorgeschlagenen Wikipediaartikel:')
print(suchergebnisse)

**Aufgabe:** Spielen sie etwas mit den obigen Zellen, in dem sie z.B. den Fragetext oder die Anzahl der gesuchten Artikelvorschläge verändern und die Zellen nochmals ausführen. Schauen sie sich an, wie sich die Liste mit Ergebnisvorschlägen verändert und überlegen sie, wie gut die einzelnen Artikel zu ihrer Frage passen. __Dokumentieren__ sie interessante Fragen und Ergebnisse in dem entsprechenden __Miro-Board__. __Überlegen__ sie sich auch, ob es eventuell sinnvoll sein könnte, die Zeichenkette mit der Frage schon etwas vorzuverarbeiten, bevor sie diese der Wikipedia-Suchfunktion übergeben.

## Der Plan

Um unsere Frage zu beantworten, wollen wir nun wie folgt vorgehen:

- Wir nehmen den ersten (das ist der von der Wikipedia-Suche als am besten passend bewertete) Artikel aus der Liste der Wikipedia-Suchergebnisse
- Falls der Artikel zu lang ist, als dass der Transformer ihn auf einmal verarbeiten könnte, zerlegen wir den Artikel in Transformer-gerechte Abschnitte
- Dann präsentieren wir dem fertig nachtrainierten Frage-Antwort-Netzwerk, das wir auch schon im letzten Notebook verwendet haben, den Artikel bzw. die einzelnen Abschnitte und geben die Antworten aus, die das Netzwerk darin findet.

Mit den Titeln, die in der Liste der Suchergebnisse gespeichert sind, können wir uns nun zunächst den Text des ersten Artikels auf der Liste ausgeben lassen.

In [ ]:
# wiki.page gibt eine Struktur zurück, die
# eine gesamte Wikipedia-Seite repräsentiert.
# Welche Seite zurückgegeben wird bestimmt man,
# in dem man einen String mit dem Titel
# eines Wikipedia-Artikels übergibt.
# In unserem Fall übergeben wir den ersten
# Titel aus der Liste unserer Suchergebnisse
# und speichern die Seitenstruktur in 
# der Variable ganze_seite
ganze_seite = wiki.page(suchergebnisse[0])

# Auf den Textinhalt der Seite kann man
# über den Zusatz '.content' zugreifen.
seiteninhalt = ganze_seite.content

print('Artikelname:')
print(suchergebnisse[0])

print('')

print('Artikeltext:')
print(seiteninhalt)

Wie sie sehen erhalten sie mit den Befehlen in der vorausgehenden Zelle den Seiteninhalt (ohne Abbildungen, Tabellen, ...) als eine zusammenhängende Zeichenkette.

## Laden des bereits nachtrainierten Frage-Antwort-Modells

Wie im vorherigen Notebook werden wir wieder das gleiche [fertig nachtrainierte deutsche Frage-Antwort-Modell](https://huggingface.co/Sahajtomar/GBERTQnA) nutzen. Wie bereits erwähnt basiert es auf einem [deutschen BERT Modell](https://huggingface.co/deepset/gbert-large)-Modell, welches auf dem deutschen Teil des [MLQA-Frage-Antwort-Datensatzes](https://github.com/facebookresearch/MLQA) finegetuned und auf dem deutschen Teil des [XQuAD-Datensatzes](https://github.com/deepmind/xquad) validiert wurde. 

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

# Lade und initialisiere das nachtrainierte deutsche Frage-Antwort-Modell https://huggingface.co/Sahajtomar/GBERTQnA.
tokenizer = AutoTokenizer.from_pretrained("Sahajtomar/GBERTQnA")
model = AutoModelForQuestionAnswering.from_pretrained("Sahajtomar/GBERTQnA")

Wir laden das Modell bereits an dieser Stelle, da wir den Tokenizer des Modells benutzen wollen, um die einzelnen Textabschnitte gleich in die entsprechenden Token-Indizes zu übersetzen, so wie wir es auch in dem vorherigen Notebook getan haben.

Zunächst übersetzten wir die Frage und den gesamten Text in eine Liste von Token-Indizes.

In [ ]:
inputs = tokenizer.encode_plus( frage, # Frage
                                seiteninhalt, # Gesamter Artikeltext
                                add_special_tokens=True, # Fügt am Anfang ein CLS-Token
                                # und zwischen Frage und Artikeltext, sowie am Ende, ein
                                # SEP-Token ein
                                return_tensors="pt", # Gibt die Token-Liste im
                                # PyTorch-Format-zurück
                                padding=False # WICHTIG: Wir füllen die Token-Liste
                                # hier **NICHT** mit PAD-Tokens bis zur Maximallänge
                                # auf, da wir GENAU WISSEN WOLLEN, WIE LANGE
                                # der gesamte Input ist
                               )

# Die nächste Zeile gibt uns nur die Liste der 
# Input-Token-Indizes zurück
input_ids = inputs["input_ids"].tolist()[0]

Nun schauen wir zunächst, wie lange die komplette Token-Indizes-Liste ist, die wir dem Netzwerk übergeben möchten.

In [ ]:
print("Anzahl der Token-Indizes in der Input-Liste:")
print(len(input_ids))

Nun können wir unser Modell auch fragen, was die maximale Länge einer Input-Token-Liste ist, die es verarbeiten kann.

In [ ]:
max_len = model.config.max_position_embeddings

print('Maximale Länge, die unser Modell verarbeiten kann:')
print(max_len)

In vielen Fällen (so wie in dem Beispiel-Artikel zur Bundesrepublik Deutschland) ist die Token-Sequenz, welche die Frage und den gesamten Artikeltext beinhaltet, um einiges länger als die 512-Tokens, die die meisten BERT-Modelle verarbeiten können (inklusive dem von uns genutzten Modell, s. **max_len**).

In der folgenden, langen Codezelle werden wir zunächst überprüfen, ob die Länge unserer Input-Liste größer ist als die maximale Anzahl an Tokens, die unser Frage-Antwort-Modell verarbeiten kann. Falls ja, zerlegen wir den Input in mundgerechte Stücke und speichern die einzelnen Stücke in einer Liste (__chunked_inputs__), falls nein verpacken wir den Input direkt in eine Liste (die in diesem Fall nur einen Eintrag enthält).

In [ ]:
# Die Funktion ceil rundet eine Zahl zur nächsten ganzen Zahl auf
# also ceil(6.2) gibt z.B. 7 zurück, ceil(8.6) gibt 9 zurück usw...
from math import ceil

## Wir müssen nur etwas tun, wenn die Länger unserer
## Input-Token-Liste len(input_ids) die maximale
## Anzahl an Input-Ids überschreitet, die unser Modell
## auf einmal verarbeiten kann (max_len)
if len(input_ids) > max_len: 
    
    # Die vom Tokenizer erzeugten Dictionaries erhalten immer drei Keys:
    #
    # 1.) input_ids: Das sind die Listen der Token-Indizes, die wir auch weiter
    #                oben betrachtet haben.
    #
    # 2.) token_type_ids: Diese Liste enthält für jedes Token einen Wert, der
    #                     angibt, ob das entsprechende Token zum ersten
    #                     Teil (Frage) oder zweiten Teil (Text) des Inputs
    #                     gehört.
    #
    # 3.) attention_mask: Diese Liste enthält die momentanen Gewichte des
    #                     Aufmerksamkeitsmechanismus (s. z.B.:
    #                     http://jalammar.github.io/illustrated-transformer/)
    #
    
    # token_type_ids wird automatisch vom Tokenizer erzeugt und
    # enthält 0 für alle Tokens, die zur Frage gehören
    # und 1 für alle Tokens, die zum Text gehören.
    # Damit können wir uns eine Maske erzeugen, die
    # nur die Tokens auswählt, für die token_type_ids
    # kleiner ist als 1 (.lt(...) steht für "less than")
    frage_maske = inputs['token_type_ids'].lt(1)
    
    # Mit dieser Maske können wir zunächst alle Tokens
    # auswählen, die zur Frage gehören
    frage_input_ids = torch.masked_select(inputs['input_ids'], frage_maske)
    frage_token_type_ids = torch.masked_select(inputs['token_type_ids'], frage_maske)
    frage_attention_mask = torch.masked_select(inputs['attention_mask'], frage_maske)
    
    
    # Länge der Frage-Token-ID-Liste
    print('Anzahl Fragetokens: %d\n' % len(frage_input_ids))
    # Indizes in Tokens übersetzen
    # und ausgeben
    print('Fragetokens:')
    print(tokenizer.convert_ids_to_tokens(frage_input_ids))
    
    # Nur Kontext-Tokens auswählen
    # Dabei schneiden wir das [SEP]-Token am Ende des Textes ab (mit [:-1])
    text_input_ids = torch.masked_select(inputs['input_ids'], ~frage_maske)[:-1]
    text_token_type_ids = torch.masked_select(inputs['token_type_ids'], ~frage_maske)[:-1]
    text_attention_mask = torch.masked_select(inputs['attention_mask'], ~frage_maske)[:-1]
    # Länge der Text-Token-ID-Liste
    print('\nAnzahl Kontexttokens: %d\n' % len(text_input_ids))
           
    # Wie viele Tokens bleiben noch übrig, wenn man von der
    # maximalen Länge, die das Netzwerk verarbeiten
    # kann, die Länge der Frage abzieht (und noch
    # ein Token für das SEP-Token freihält, das
    # ganz am Ende des Netzwerk-Inputs stehen
    # muss)?
    # Das ist die Größe eines Bissens ("Chunks")
    # aus dem Artikeltext, den das Netzwerk auf
    # einmal verarbeiten kann.
    chunk_size = max_len - frage_maske.sum() - 1
    
    print('Maximale Anzahl an Texttokens pro Textabschnitt: %d' % chunk_size)
    
    # Anzahl der Chunks:
    n_chunks = int(ceil(len(text_input_ids) / chunk_size))
    
    print('Anzahl der Textabschnitte: %d\n' % n_chunks)
    
    # In dieser Liste werden wir die einzelnen Input-Zeichenketten
    # ([CLS] Frage [SEP] Kontextabschnitt [SEP]) speichern, um diese
    # später nacheinander dem trainierten BERT-Modell zu präsentieren.
    chunked_inputs = []
    
    for i in range(n_chunks):
        
        # i zählt von 0 bis zur Anzahl der Chunks - 1
        # chunk_size ist die Länge eines einzelnen
        # Chunks aus dem Artikeltext
        
        # Wir berechnen, wo der Textabschnitt, den wir
        # in diesem Durchlauf extrahieren wollen,
        # anfängt und aufhört.
        start_index = i*chunk_size        
        end_index = i*chunk_size + chunk_size
        
        # Wenn das Ende des Textabschnittes über 
        # das Ende des Textes hinausgeht, wird 
        # es auf das Ende des Textes gesetzt.
        # Dies ist nur für den letzten Textabschnitt
        # relevant, der eventuell kürzer als
        # chunk_size ist.
        if end_index > len(text_input_ids) - 1:
            end_index = len(text_input_ids) - 1
        
        # Hier wird der entsprechende Input erzeugt,
        # in dem die Frage-Tokens, die Text-Tokens,
        # die dem entsprechenden Textabschnitt entsprechen 
        # (von start_index bis end_index - 1) und ein
        # [SEP]-Token (torch.tensor([103])) aneinander 
        # gehängt werden.
        
        chunk = {
            
            'input_ids': torch.cat((frage_input_ids, text_input_ids[start_index:end_index], torch.tensor([103]))).unsqueeze(dim=0),
            'token_type_ids': torch.cat((frage_token_type_ids, text_token_type_ids[start_index:end_index], torch.tensor([1]))).unsqueeze(dim=0),
            'attention_mask': torch.cat((frage_attention_mask, text_attention_mask[start_index:end_index], torch.tensor([1]))).unsqueeze(dim=0)     
            
        }
        
        # Der gerade erzeugte Input-Chunk wird in der Liste gespeichert.
        chunked_inputs.append(chunk)   
        
        print('Chunk Nr.: %d' % i)
        print('Length of Chunk: %d\n' % chunk['input_ids'].shape[1])   
        
# Falls der Input so kurz ist, dass er gar nicht aufgeteilt
# werden muss, packen wir ihn direkt als einzigen Eintrag
# in die chunked_inputs-Liste.
else:
    chunked_inputs = [inputs]
      

In der folgenden Zelle werden wir nun jeden Input-Chunk aus der chunked_inputs-Liste einmal an das Frage-Antwort-Netzwerk übergeben. Dann werden wir die vom Modell generierter start_logits und end_logits Listen, genau wie im vorherigen Notebook, benutzen, um die Antwort aus jedem einzelnen Chunk zu extrahieren und auszugeben.

In [ ]:
# Wir geben noch einmal die Frage aus (zur Erinnerung)
print("Question: " + frage)

# Wir iterieren über die Liste mit den "mundgerechten"
# Inputs für das Transformer-Modell
for i in range(len(chunked_inputs)):
        
    # Wir wählen den Input aus, den wir in diesem
    # Durchlauf verarbeiten möchten
    chunk = chunked_inputs[i]
    
    # Wir übergeben den Input an das nachtrainierte
    # Frage-Antwort-Sprachmodell    
    output = model(**chunk)

    # Wir extrahieren die Listen mit den Start-Logits
    # und den End-Logits aus dem Netzwerkoutput
    # und berechnen damit den wahrscheinlichsten 
    # Start-Index und den wahrscheinlichsten End-Index
    # für die Antwort.
    start_index = torch.argmax(output['start_logits'])
    end_index = torch.argmax(output['end_logits'])
    
    # Wir wählen die entsprechenden Token-IDs aus der
    # Token-ID-Liste des entsprechenden Chunks aus 
    # und übersetzen sie zurück in eine normale Zeichenkette
    chunk_ids = chunk["input_ids"].tolist()[0] # Token-ID-Liste des gerade verarbeiteten Chunks
    antwort = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(chunk_ids[start_index:end_index+1]))

    # Wir geben für jeden Chunk zunächst dessen Nummer aus
    print("Chunk %d:" % i)
    
    # Falls sowohl start_index, als auch end_index 0 sind,
    # bedeutet das, dass unser Modell in dem entsprechenden
    # Abschnitt keine Antwort gefunden hat (entspricht der
    # Rückgabe des [CLS]-Tokens am Anfang der Inputsequenz)
    if start_index == 0 and end_index == 0:
        print('No answer found in this chunk!')        
    # Sonst geben wir die entsprechende Antwortzeichenkette aus
    else:        
        print("Answer: " + antwort)
        print("Start: %d End: %d" % (start_index, end_index))
    print("")

Phew! Zugegebener Maßen waren das die längsten und komplexesten Codeblöcke heute. Dafür haben sie sich jetzt einmal durch den detaillierten Code gekämpft, den es braucht, um ein bereits vorhandenes und fertig nachtrainiertes Frage-Antwort-Netzwerk in der Praxis einzusetzen.

Um die Grundlegenden Konzepte nochmals zu wiederholen, und ihnen einmal vor Augen zu führen, warum es sinnvoll sein kann, einzelne Code-Abschnitte in selbst-geschriebene Funktionen zu verpacken, stellen ihnen die folgenden zwei Zellen entsprechende Hilfsfunktionen vor, welche die zwei folgenden Funktionen erfüllen:

- Heraussuchen von Wikipedia-Artikeln anhand einer gegebenen Suchanfrage
- Zerlegen und verarbeiten einer gegebenen Zeichenkette (d.h. eines einzelnen Artikels oder Textabschnittes) mit einem Frage-Antwort-Netzwerk.

## Hilfsfunktionen

In den vorhergehenden Zellen haben sie gesehen und gelernt, wie man das __wikipedia__-Paket benutzen kann, um aus Python heraus auf die Wikipediasuche zuzugreifen und den Text der entsprechenden Artikel als Zeichenkette zu erhalten.

Des weiteren haben sie gelernt, wie man Artikel, die immer noch zu lang für die Verarbeitung durch ein gegebenes Transformer-Modell sind, in mundgerechte Stücke ("chunks") zerlegen und diese einzeln an das Transformernetzwerk übergeben kann.

Da der obige Code doch recht sperrig ist, haben wir für sie zwei Hilfsfunktionen erstellt, die die entsprechende Funktionalität gekapselt zur Verfügung stellen. Diese wollen wir ihnen hier noch einmal vorstellen.

### Wikipedia-Suche

Die Funktionen des __wikipedia__-Pythonpaketes sind zwar schon sehr schön gekapselt, wir haben sie jedoch noch etwas kompakter in eine einzige Funktion verpackt. Der Funktion __get_wikipedia_articles__ können sie einen Fragetext übergeben, sowie (optional) die Anzahl __n__ der Artikel, die zu dieser Frage herausgesucht werden sollen.

In [ ]:
from transformer_helper_functions import get_wikipedia_articles

frage = 'Wie viele Bundesländer hat die Bundesrepublik Deutschland?'

titel, texte = get_wikipedia_articles(frage, n = 3)

Die Funktion gibt zwei Listen zurück, welche jeweils die __titel__ und __texte__ der gefundenen Wikipedia-Artikel enthalten. Wir können die Funktion __len__ benutzen, um zu schauen, wie viele Artikel zurückgegeben wurden, in dem wir die Länge einer der beiden (gleichlangen) Listen bestimmen.

In [ ]:
print('Anzahl der gefundenen Artikel:')
print(len(titel))

Wir können außerdem über die Listen iterieren, um die Titel und Texte auszugeben.

In [ ]:
for i in range(len(titel)):
    print('Suchergebnis Nr. %d:' % i)
    print('Titel: ' + titel[i])
    print('Text: ' + texte[i])
    print('')

### Chunking und Aufruf des Transformer-Modells

Wir haben nun in der Datei transformer_helper_functions.py zudem die Hilfsfunktion __chunkify_and_get_answers__ zur Verfügung gestellt. Dieser Funktion können sie eine Frage, einen Text aus dem die Antwort auf diese Frage extrahiert werden soll, sowie ein auf Frage-Antwort-Tasks nachtrainiertes BERT-Modell (natürlich mit dem zugehörigen Tokenizer) übergeben. Die Funktion zerlegt dann den Text in "mundgerechte" Stücke ("chunks"), die nacheinander dem Netzwerk übergeben werden. Für jeden chunk wird die entsprechende Antwort des Netzwerkes ausgegeben. Falls das Netzwerk in einem Chunk keine Antwort findet, gibt es für diesen Abschnitt '[CLS]' als Antwort aus.

In [ ]:
from transformer_helper_functions import chunkify_and_get_answers

frage = 'Wann promovierte Alan Turing?'

text = """Alan Mathison Turing OBE,[2] FRS[3] [ˈælən ˈmæθɪsən ˈtjʊəɹɪŋ] (* 23. Juni 1912 in London; † 7. Juni 1954 in Wilmslow, Cheshire) war ein britischer Logiker, Mathematiker, Kryptoanalytiker und Informatiker. Er gilt heute als einer der einflussreichsten Theoretiker der frühen Computerentwicklung und Informatik. Turing schuf einen großen Teil der theoretischen Grundlagen für die moderne Informations- und Computertechnologie. Als richtungsweisend erwiesen sich auch seine Beiträge zur theoretischen Biologie.
Das von ihm entwickelte Berechenbarkeitsmodell der Turingmaschine bildet eines der Fundamente der Theoretischen Informatik. Während des Zweiten Weltkrieges war er maßgeblich an der Entzifferung der mit der deutschen Rotor-Chiffriermaschine Enigma verschlüsselten deutschen Funksprüche beteiligt. Der Großteil seiner Arbeiten blieb auch nach Kriegsende unter Verschluss.
Turing entwickelte 1953 eines der ersten Schachprogramme, dessen Berechnungen er mangels Hardware selbst vornahm. Nach ihm benannt sind der Turing Award, die bedeutendste Auszeichnung in der Informatik, sowie der Turing-Test zum Überprüfen des Vorhandenseins von künstlicher Intelligenz.[4]
Im März 1952 wurde Turing wegen seiner Homosexualität, die damals noch als Straftat verfolgt wurde, zur chemischen Kastration verurteilt.[5][6] Turing erkrankte in Folge der Hormonbehandlung an einer Depression und starb etwa zwei Jahre später durch Suizid. Im Jahr 2009 sprach der damalige britische Premierminister Gordon Brown eine offizielle Entschuldigung im Namen der Regierung für die „entsetzliche Behandlung“ Turings aus und würdigte dessen „außerordentliche Verdienste“ während des Krieges; eine Begnadigung wurde aber noch 2011 trotz einer Petition abgelehnt. Am Weihnachtsabend, dem 24. Dezember 2013, sprach Königin Elisabeth II. postum ein „Royal Pardon“ (Königliche Begnadigung) aus.[7][8][9][10]
Alan Turings Vater, Julius Mathison Turing, war britischer Beamter beim Indian Civil Service. Er und seine Frau Ethel Sara (geborene Stoney) wünschten, dass ihre Kinder in Großbritannien aufwüchsen.[11] Deshalb kehrte die Familie vor Alans Geburt aus Chatrapur, damals Britisch-Indien, nach London-Paddington zurück, wo Alan Turing am 23. Juni 1912 zur Welt kam. Da der Staatsdienst seines Vaters noch nicht beendet war, reiste dieser im Frühjahr 1913 erneut nach Indien, wohin ihm seine Frau im Herbst folgte. Turing und sein älterer Bruder John wurden nach St. Leonards-on-the-Sea, Hastings, in die Familie eines pensionierten Obersts und dessen Frau in Pflege gegeben. In der Folgezeit pendelten die Eltern zwischen England und Indien, bis sich Turings Mutter 1916 entschied, längere Zeit in England zu bleiben, und die Söhne wieder zu sich nahm.
Schon in früher Kindheit zeigte sich die hohe Begabung und Intelligenz Turings. Es wird berichtet, dass er sich innerhalb von drei Wochen selbst das Lesen beibrachte und sich schon früh zu Zahlen und Rätseln hingezogen fühlte.[11]
Im Alter von sechs Jahren wurde Turing auf die private Tagesschule St. Michael’s in St. Leonards-on-the-Sea geschickt, wo die Schulleiterin frühzeitig seine Begabung bemerkte. 1926, im Alter von 14 Jahren, wechselte er auf die Sherborne School in Dorset. Sein erster Schultag dort fiel auf einen Generalstreik in England. Turing war jedoch so motiviert, dass er die 100 Kilometer von Southampton zur Schule allein auf dem Fahrrad zurücklegte und dabei nur einmal in der Nacht an einer Gaststätte Halt machte; so berichtete jedenfalls die Lokalpresse.
Turings Drang zur Naturwissenschaft traf bei seinen Lehrern in Sherborne auf wenig Gegenliebe; sie setzten eher auf Geistes- als auf Naturwissenschaften. Trotzdem zeigte Turing auch weiterhin bemerkenswerte Fähigkeiten in den von ihm geliebten Bereichen. So löste er für sein Alter fortgeschrittene Aufgabenstellungen, ohne zuvor irgendwelche Kenntnisse der elementaren Infinitesimalrechnung erworben zu haben.
Im Jahr 1928 stieß Turing auf die Arbeiten Albert Einsteins. Er verstand sie nicht nur, sondern entnahm einem Text selbständig Einsteins Bewegungsgesetz, obwohl dieses nicht explizit erwähnt wurde.

Turings Widerstreben, für Geisteswissenschaften genauso hart wie für Naturwissenschaften zu arbeiten, hatte zur Folge, dass er einige Male durch die Prüfungen fiel. Weil dies seinen Notendurchschnitt verschlechterte, musste er 1931 auf ein College zweiter Wahl gehen, das King’s College, Cambridge, entgegen seinem Wunsch, am Trinity College zu studieren. Er studierte von 1931 bis 1934 unter Godfrey Harold Hardy (1877–1947), einem respektierten Mathematiker, der den Sadleirian Chair in Cambridge innehatte, das zu der Zeit ein Zentrum der mathematischen Forschung war.
In seiner für diesen Zweig der Mathematik grundlegenden Arbeit On Computable Numbers, with an Application to the „Entscheidungsproblem“ (28. Mai 1936) formulierte Turing die Ergebnisse Kurt Gödels von 1931 neu. Er ersetzte dabei Gödels universelle, arithmetisch-basierte formale Sprache durch einen einfachen gedanklichen Mechanismus, eine abstrakt-formale Zeichenketten verarbeitende mathematische Maschine, die heute unter dem Namen Turingmaschine bekannt ist. („Entscheidungsproblem“ verweist auf eines der 23 wichtigsten offenen Probleme der Mathematik des 20. Jahrhunderts, vorgestellt von David Hilbert 1900 auf dem 2. Internationalen Mathematiker-Kongress in Paris [„Hilbertsche Probleme“].) Turing bewies, dass solch ein Gerät in der Lage ist, „jedes vorstellbare mathematische Problem zu lösen, sofern dieses auch durch einen Algorithmus gelöst werden kann“.
Turingmaschinen sind bis zum heutigen Tag einer der Schwerpunkte der Theoretischen Informatik, nämlich der Berechenbarkeitstheorie. Mit Hilfe der Turingmaschine gelang Turing der Beweis, dass es keine Lösung für das „Entscheidungsproblem“ gibt. Er zeigte, dass die Mathematik in gewissem Sinne unvollständig ist, weil es allgemein keine Möglichkeit gibt, festzustellen, ob eine beliebige, syntaktisch korrekt gebildete mathematische Aussage beweisbar oder widerlegbar ist. Dazu bewies er, dass das Halteproblem für Turingmaschinen nicht lösbar ist, d. h., dass es nicht möglich ist, algorithmisch zu entscheiden, ob eine Turingmaschine, angesetzt auf eine Eingabe (initiale Bandbelegung), jemals zum Stillstand kommen wird, das heißt die Berechnung terminiert. Turings Beweis wurde erst nach dem von Alonzo Church (1903–1995) mit Hilfe des Lambda-Kalküls geführten Beweis veröffentlicht; unabhängig davon ist Turings Arbeit beträchtlich einfacher und intuitiv zugänglich. Auch war der Begriff der „Universellen (Turing-) Maschine“ neu, einer Maschine, welche jede beliebige andere Turing-Maschine simulieren kann. Die Eingabe für diese Maschine ist also ein verschlüsseltes Programm, das von der universellen Maschine interpretiert wird, und der Startwert, auf den es angewendet werden soll.
Alle bis heute definierten Berechenbarkeitsbegriffe haben sich (bis auf die Abbildung von Worten auf Zahlen und umgekehrt) als äquivalent erwiesen.
1938 und 1939 verbrachte Turing zumeist an der Princeton University und studierte dort unter Alonzo Church. 1938 erwarb er den Doktorgrad in Princeton. Seine Doktorarbeit führte den Begriff der „Hypercomputation“ ein, bei der Turingmaschinen zu sogenannten Orakel-Maschinen erweitert werden. So wurde das Studium von nicht-deterministisch lösbaren Problemen ermöglicht.
"""
answers = chunkify_and_get_answers(frage, text, tokenizer, model)

Die Funktion gibt eine Liste __answers__ zurück, die für jeden Chunk des Informationstextes eine Antwort enthält. Um zu zählen, wie viele es sind, benutzen wir wieder __len__, um die Länge der Antwortliste zu bestimmen.

In [ ]:
print('Der Informationstext wurde in folgende Anzahl an Chunks zerlegt:')
print(len(answers))

Wir können nun über die __answers__-Liste iterieren und die einzelnen Antworten ausgeben.

In [ ]:
for i in range(len(answers)):
    
    print('Antwort Nr. %d:' % i)
    print(answers[i])

Hier sehen sie, wie nützlich es sein kann, längere Codeblöcke in Hilfsfunktionen auszulagern, und wie sehr der Code so an Übersichtlichkeit und Lesbarkeit gewinnt.